Uma tabela dinâmica é uma maneira de resumir dados em um DataFrame para uma finalidade específica. Faz uso pesado da função de agregação. Uma tabela dinâmica é em si um DataFrame, onde as linhas representam uma variável em que você está interessado, as colunas outra e algum valor agregado da célula. Uma tabela dinâmica também tende a incluir valores marginais, que são as somas de cada coluna e linha. Isso permite que você veja a relação entre duas variáveis ​​apenas de relance.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Aqui temos o conjunto de dados do Times Higher Education World University Ranking, que é uma das medidas 
# universitárias mais influentes. Vamos importar o conjunto de dados e ver como fica
df = pd.read_csv('/content/cwurData.csv')
df.head()

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012


In [10]:
# Aqui podemos ver a classificação de cada instituição, país, qualidade da educação, outras métricas e pontuação geral.
# Digamos que queremos criar uma nova coluna chamada Rank_Level, onde as instituições com ranking mundial 1-100 são
# categorizado como primeiro nível e aqueles com ranking mundial 101 - 200 são de segundo nível, ranking 201 - 300 são
# terceiro nível, depois de 301 é outras universidades de topo.

def create_category(ranking):
  if (ranking >=1) & (ranking <=100):
    return 'First Tier'
  elif (ranking >=101) & (ranking <=200):
    return 'Second Tier'
  elif (ranking>=201) & (ranking <=300):
    return 'Third Tier'
  return 'Other top University'

#agora vamos aplicar nossa função a coluna world_rank para criar uma nova series 
df['Rank_Level'] = df['world_rank'].apply(lambda x: create_category(x))
df.head(500)

,world_rank,institution,country,national_rank,quality_of_education,alumni_employment,quality_of_faculty,publications,influence,citations,broad_impact,patents,score,year,Rank_Level
0,1,Harvard University,USA,1,7,9,1,1,1,1,NaN,5,100.00,2012,First Tier
1,2,Massachusetts Institute of Technology,USA,2,9,17,3,12,4,4,NaN,1,91.67,2012,First Tier
2,3,Stanford University,USA,3,17,11,5,4,2,2,NaN,15,89.50,2012,First Tier
3,4,University of Cambridge,United Kingdom,1,10,24,4,16,16,11,NaN,50,86.17,2012,First Tier
4,5,California Institute of Technology,USA,4,2,29,7,37,22,22,NaN,18,85.21,2012,First Tier
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,296,"Washington State University, Pullman",USA,115,355,267,122,293,221,310,295.0,170,46.40,2014,Third Tier
496,297,Technical University of Berlin,Germany,24,80,276,210,353,432,406,372.0,115,46.39,2014,Third Tier
497,298,University of South Carolina - Columbia,USA,116,355,440,182,262,353,130,266.0,737,46.37,2014,Third Tier
498,299,Leipzig University,Germany,25,308,478,201,287,268,800,224.0,307,46.36,2014,Third Tier


In [11]:
# Uma tabela dinâmica nos permite dinamizar uma dessas colunas em um novo cabeçalho de coluna e compará-la com
# outra coluna como índices de linha. Digamos que queremos comparar o nível de classificação com o país das universidades
# e queremos comparar em termos de pontuação geral

# Para fazer isso, informamos ao Pandas que queremos que os valores sejam Score, e index seja o país e as colunas sejam
# os níveis de classificação. Em seguida, especificamos que a função de agregação, e aqui usaremos a média NumPy para obter a
# classificação média para universidades naquele país
df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean]).head()

mean                                            
Rank_Level First Tier Other top University Second Tier Third Tier
country                                                          
Argentina         NaN            44.672857         NaN        NaN
Australia     47.9425            44.645750     49.2425  47.285000
Austria           NaN            44.864286         NaN  47.066667
Belgium       51.8750            45.081000     49.0840  46.746667
Brazil            NaN            44.499706     49.5650        NaN

In [12]:
# Podemos ver um dataframe hierárquico onde o índice, ou linhas, são por país e as colunas têm dois
# níveis, o nível superior indicando que o valor médio está sendo usado e o segundo nível sendo nossas classificações.
# neste exemplo temos apenas uma variável, a média, que estamos olhando, então não precisamos de um
# índice hierárquico.

# Notamos que existem alguns valores NaN, por exemplo, a primeira linha, Argentia. Os valores de NaN indicam que
# A Argentia tem apenas observações na categoria "Outras principais universidades"

In [14]:
# Agora, as tabelas dinâmicas não estão limitadas a uma função que você pode querer aplicar. Você pode passar 
# o parametro aggfunc, que é uma lista das diferentes funções a serem aplicadas, e os pandas fornecera a você
# o resultado usando nomes de colunas hierárquicas. Vamos tentar a mesma consulta, mas passe a função max() também

df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max]).head()

mean                                                   amax  \
Rank_Level First Tier Other top University Second Tier Third Tier First Tier   
country                                                                        
Argentina         NaN            44.672857         NaN        NaN        NaN   
Australia     47.9425            44.645750     49.2425  47.285000      51.61   
Austria           NaN            44.864286         NaN  47.066667        NaN   
Belgium       51.8750            45.081000     49.0840  46.746667      52.03   
Brazil            NaN            44.499706     49.5650        NaN        NaN   

                                                        
Rank_Level Other top University Second Tier Third Tier  
country                                                 
Argentina                 45.66         NaN        NaN  
Australia                 45.97       50.40      47.47  
Austria                   46.29         NaN      47.78  
Belgium                   46.21       49.73      47.14  
Brazil                    46.08       49.82        NaN

In [15]:
# Uma tabela dinâmica é apenas um dataframe multinível, e podemos acessar séries ou células no dataframe de maneira semelhante
# como fazemos para um dataframe regular.

#vamos criar um novo dataframe com o exemplo anterior 
new_df=df.pivot_table(values='score', index='country', columns='Rank_Level', aggfunc=[np.mean, np.max], 
               margins=True)

#vamos olhar o index 
print(new_df.index)
#e também vamos olhar as colunas 
print(new_df.columns)

Index(['Argentina', 'Australia', 'Austria', 'Belgium', 'Brazil', 'Bulgaria',
       'Canada', 'Chile', 'China', 'Colombia', 'Croatia', 'Cyprus',
       'Czech Republic', 'Denmark', 'Egypt', 'Estonia', 'Finland', 'France',
       'Germany', 'Greece', 'Hong Kong', 'Hungary', 'Iceland', 'India', 'Iran',
       'Ireland', 'Israel', 'Italy', 'Japan', 'Lebanon', 'Lithuania',
       'Malaysia', 'Mexico', 'Netherlands', 'New Zealand', 'Norway', 'Poland',
       'Portugal', 'Puerto Rico', 'Romania', 'Russia', 'Saudi Arabia',
       'Serbia', 'Singapore', 'Slovak Republic', 'Slovenia', 'South Africa',
       'South Korea', 'Spain', 'Sweden', 'Switzerland', 'Taiwan', 'Thailand',
       'Turkey', 'USA', 'Uganda', 'United Arab Emirates', 'United Kingdom',
       'Uruguay', 'All'],
      dtype='object', name='country')
MultiIndex([('mean',           'First Tier'),
            ('mean', 'Other top University'),
            ('mean',          'Second Tier'),
            ('mean',           'Third Tier'),

In [17]:
# Podemos ver que as colunas são hierárquicas. Os índices de coluna de nível superior têm duas categorias: média e máxima, e
# os índices de coluna de nível inferior têm quatro categorias, que são os quatro níveis de classificação. Como poderíamos consultar isso
# se quisermos obter as pontuações médias dos níveis das universidades de primeiro nível em cada país? Nós apenas precisaríamos
# para fazer duas projeções de dataframe, a primeira para a média, depois a segunda para a camada superior
new_df['mean']['First Tier'].head()

country
Argentina        NaN
Australia    47.9425
Austria          NaN
Belgium      51.8750
Brazil           NaN
Name: First Tier, dtype: float64

In [19]:
#podemos confirmar que a saida é uma series 
type(new_df['mean']['First Tier'])

pandas.core.series.Series

In [20]:
# E se quisermos encontrar o país que tem a pontuação média máxima no nível First Tier Top University?
# Podemos usar a função idxmax().
new_df['mean']['First Tier'].idxmax()

'United Kingdom'

In [22]:
#caso queiramos obter uma forma diferente da nossa tabela dinamica podemos fazer o indice de coluna mais baixo virar o indice de linha 
# e o inverso 
new_df.head()

mean                                                         \
Rank_Level First Tier Other top University Second Tier Third Tier        All   
country                                                                        
Argentina         NaN            44.672857         NaN        NaN  44.672857   
Australia     47.9425            44.645750     49.2425  47.285000  45.825517   
Austria           NaN            44.864286         NaN  47.066667  45.139583   
Belgium       51.8750            45.081000     49.0840  46.746667  47.011000   
Brazil            NaN            44.499706     49.5650        NaN  44.781111   

                 amax                                                     
Rank_Level First Tier Other top University Second Tier Third Tier    All  
country                                                                   
Argentina         NaN                45.66         NaN        NaN  45.66  
Australia       51.61                45.97       50.40      47.47  51.61  
Austria           NaN                46.29         NaN      47.78  47.78  
Belgium         52.03                46.21       49.73      47.14  52.03  
Brazil            NaN                46.08       49.82        NaN  49.82

In [23]:
# Agora vamos tentar empilhar, isso deve mover a coluna mais baixa, então as camadas do ranking da universidade, para
# a linha mais interna
new_df=new_df.stack()
new_df.head()

mean   amax
country   Rank_Level                            
Argentina Other top University  44.672857  45.66
          All                   44.672857  45.66
Australia First Tier            47.942500  51.61
          Other top University  44.645750  45.97
          Second Tier           49.242500  50.40

In [24]:
# Isso parece restaurar nosso dataframe à sua forma original. O que você acha que aconteceria se desempilhamos duas vezes seguidas?
new_df.unstack().unstack().head()

      Rank_Level  country  
mean  First Tier  All          58.350675
                  Argentina          NaN
                  Australia    47.942500
                  Austria            NaN
                  Belgium      51.875000
dtype: float64

In [ ]:
# Na verdade, acabamos desempilhando até uma única coluna, então um objeto de série é retornado. esta
# coluna é apenas um "valor", cujo significado é denotado pelo índice hierárquico de operação, classificação e
# país.

Então isso é tabelas dinâmicas. Esta foi uma descrição bastante curta, mas eles são incrivelmente úteis ao lidar com dados numéricos, especialmente se você estiver tentando resumir os dados de alguma forma. Você criará regularmente novas tabelas dinâmicas em fatias de dados, quer esteja explorando os dados por conta própria ou preparando dados para outros relatarem. E, claro, você pode passar qualquer função que desejar para a função agregada, incluindo aquelas que você mesmo define.